In [9]:
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_preprocessing.image import ImageDataGenerator
#from tensorflow.keras.optimizers import Adam
from keras.optimizers import Adam
#from tensorflow.keras.preprocessing.image import img_to_array
from keras_preprocessing.image import img_to_array
# from tensorflow.keras.utils import to_categorical, plot_model
from keras.utils import to_categorical,plot_model
# from tensorflow.keras.models import Sequential
from keras.models import Sequential
# from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
#from tensorflow.keras import backend as K
from keras import backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os
import glob

from sklearn.metrics import confusion_matrix,accuracy_score,f1_score

In [10]:
epochs = 100
lr = 1e-3
batch_size = 64
img_dims = (96,96,3)

data = []
labels = []

In [11]:
# load image files from the dataset
image_files = [f for f in glob.glob(r'Male_and_Female_face_dataset' + "/**/*", recursive=True) if not os.path.isdir(f)]
random.shuffle(image_files)

# converting images to arrays and labelling the categories
for img in image_files:

    image = cv2.imread(img)
    
    image = cv2.resize(image, (img_dims[0],img_dims[1]))
    image = img_to_array(image)
    data.append(image)

    label = img.split(os.path.sep)[-2] 
    if label == "woman":
        label = 1
    else:
        label = 0
        
    labels.append([label]) # [[1], [0], [0], ...]

# pre-processing
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2,
                                                  random_state=42)

trainY = to_categorical(trainY, num_classes=2) # [[1, 0], [0, 1], [0, 1], ...]
testY = to_categorical(testY, num_classes=2)

# augmenting datset 
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")

# define model
def build(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    if K.image_data_format() == "channels_first": #Returns a string, either 'channels_first' or 'channels_last'
        inputShape = (depth, height, width)
        chanDim = 1
    
    # The axis that should be normalized, after a Conv2D layer with data_format="channels_first", 
    # set axis=1 in BatchNormalization.

    model.add(Conv2D(32, (3,3), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(classes))
    model.add(Activation("sigmoid"))

    return model

# build model
model = build(width=img_dims[0], height=img_dims[1], depth=img_dims[2],
                            classes=2)

# compile the model
opt = Adam(lr=lr, decay=lr/epochs)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the model
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),
                        validation_data=(testX,testY),
                        steps_per_epoch=len(trainX) // batch_size,
                        epochs=epochs, verbose=1)




c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_10168\762342674.py:97: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),


Epoch 1/100
67/67 [==============================] - 54s 788ms/step - loss: 0.8049 - accuracy: 0.6597 - val_loss: 0.7069 - val_accuracy: 0.5618
Epoch 2/100
67/67 [==============================] - 47s 699ms/step - loss: 0.5872 - accuracy: 0.7354 - val_loss: 0.9780 - val_accuracy: 0.4972
Epoch 3/100
67/67 [==============================] - 47s 702ms/step - loss: 0.5055 - accuracy: 0.7749 - val_loss: 1.2415 - val_accuracy: 0.5360
Epoch 4/100
67/67 [==============================] - 65s 973ms/step - loss: 0.4736 - accuracy: 0.7911 - val_loss: 0.5955 - val_accuracy: 0.7113
Epoch 5/100
67/67 [==============================] - 54s 796ms/step - loss: 0.4094 - accuracy: 0.8333 - val_loss: 0.9572 - val_accuracy: 0.6236
Epoch 6/100
67/67 [==============================] - 53s 794ms/step - loss: 0.3868 - accuracy: 0.8419 - val_loss: 0.5237 - val_accuracy: 0.7703
Epoch 7/100
67/67 [==============================] - 50s 745ms/step - loss: 0.3730 - accuracy: 0.8459 - val_loss: 1.3741 - val_accuracy:

In [12]:
model.save('new_model')

INFO:tensorflow:Assets written to: my_model\assets


INFO:tensorflow:Assets written to: my_model\assets


In [36]:
y_pred = model.predict(testX,use_multiprocessing=True, workers=os.cpu_count)


accuracy_score(testY,np.round(abs(y_pred)))

34/34 [==============================] - 2s 70ms/step


0.985239852398524